# RED WINE QUALITY

`redsine-quality.ipynb`


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('/Users/gim-yujin/Desktop/TIL/ML/winequality-red.csv')

In [25]:
# 특성 살펴보기(품질 분포, 히스토그램)
#1. 상위 5개 행 확인 
print(df.head())
#데이터 기본 정보 확인 
print(df.info())
#기본 통계 요약 
print(df.describe())
# 컬럼별 고유값 개수 
print(df.nunique())
# 결측치 확인
print(df.isnull().sum())


   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  quality_labels  
0      9.4        5               0  
1      9.8

In [ ]:
# 품질과의 상관관계 (히트맵)/ 품질에 영향을 많이 주는 특성들 시각화 
# 상관계수 계산
corr = df.corr()

# 히트맵 시각화
plt.figure(figsize=(12,8))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap with Quality", fontsize=14)
plt.show()

In [ ]:
# 품질과 상관계수만 추출
quality_corr = corr["quality"].sort_values(ascending=False)
print("품질과의 상관관계:\n", quality_corr)

In [ ]:
# 절댓값 기준 상관관계 Top 3 특성 선택
top_features = quality_corr.abs().sort_values(ascending=False)[1:4].index

for feature in top_features:
    plt.figure(figsize=(6,4))
    sns.scatterplot(x=df[feature], y=df['quality'])
    sns.regplot(x=df[feature], y=df['quality'], scatter=False, color="red")
    plt.title(f"{feature} vs Quality", fontsize=13)
    plt.show()

In [ ]:
# 추가로 quality_labels 컴럼 생성 -> 0~5: 0 / 6~7: 1 / 8~10 : 2  --> 컬럼 기준으로 범주형 라벨을 만들어 분류 문제용 데이터로 변환 

# quality_labels 생성
df["quality_labels"] = df["quality"].apply(lambda x: 0 if x <= 5 else (1 if x <= 7 else 2))

print(df[["quality", "quality_labels"]].head(10))
print(df["quality_labels"].value_counts())

In [ ]:
#훈련 테스트 셋 나누기 
import pandas as pd
from sklearn.model_selection import train_test_split

# 입력 데이터 (X), 타깃 데이터 (y) 분리
X = df.drop(["quality", "quality_labels"], axis=1)  # 특성 (quality 제외)
y = df["quality_labels"]                           # 타깃 (라벨)

# 훈련/테스트 데이터 분할 (기본: 8:2 비율, 라벨 분포를 유지하기 위해 stratify 사용)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,          # 테스트 데이터 20%
    random_state=42,        # 재현 가능성 위해 시드 고정
    ## random _state를 고정하는 것은 데이터를 동일한 방식으로 분할하는 것 의미. 
    stratify=y              # 클래스 비율 유지
)

print("훈련 데이터 크기:", X_train.shape, y_train.shape)
print("테스트 데이터 크기:", X_test.shape, y_test.shape)
print("라벨 분포(훈련):\n", y_train.value_counts(normalize=True))
print("라벨 분포(테스트):\n", y_test.value_counts(normalize=True))

- random_state를 설정하지 않으면 코드를 실행할 때마다 데이터가 다르게 섞이므로, 훈련 데이터와 테스트 데이터가 매번 달라집니다. 이렇게 되면 모델의 성능 평가 결과가 실행할 때마다 조금씩 달라져서, 실험 결과를 **재현(reproduce)**하고 비교하기가 어려워집니다.

- random_state를 특정 값(예: 42)으로 설정하면 데이터가 항상 동일한 순서로 섞이게 됩니다.  이렇게 되면 코드를 몇 번이고 다시 실행하더라도 항상 같은 훈련/테스트 데이터 분할을 얻게 됩니다. 이는 다른 사람과 결과를 공유하거나, 나중에 다시 같은 실험을 수행할 때 일관된 결과를 보장해줍니다.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 데이터 불러오기

# KNN 회귀로 와인 품질 예측 
# 입력(X) , 타깃(y) 로 설정 qulity 는 그대로 사용 -> 회귀 
#KNN 은 거리기반이라 스케일링이 필수이다. 
from sklearn.neighbors import KNeighborsClassifier

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# KNN 분류기 (기본 k=5)
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_scaled, y_train)

# 예측
y_pred = knn_clf.predict(X_test_scaled)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np



# 입력(X), 타깃(y) 설정 (quality 예측)
X = df.drop("quality", axis=1)
y = df["quality"]

# 훈련/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 선형 회귀 모델
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# 예측
y_train_pred = lr.predict(X_train_scaled)
y_test_pred = lr.predict(X_test_scaled)

# 성능 평가
print("훈련셋 R²:", r2_score(y_train, y_train_pred))
print("테스트셋 R²:", r2_score(y_test, y_test_pred))
print("훈련셋 RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))
print("테스트셋 RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))

훈련셋 R²: 0.7889443175710658
테스트셋 R²: 0.8142354773202403
훈련셋 RMSE: 0.37055533838505345
테스트셋 RMSE: 0.3484224505327162




⸻

📌 1. R² (결정계수, Coefficient of Determination)

정의:
	•	R²는 모델이 실제 데이터를 얼마나 잘 설명하는지를 나타내는 지표입니다.
	•	값의 범위:
	•	1.0 → 완벽한 예측
	•	0.0 → 평균값으로만 예측한 것과 동일 (모델이 데이터 변동을 전혀 설명 못 함)
	•	음수 → 모델이 아예 평균보다 못한 경우

수식:

R^2 = 1 - \frac{SSR}{SST}
	•	SSR = 잔차 제곱합 (Sum of Squared Residuals, 예측 오차)
	•	SST = 총 제곱합 (데이터 자체의 변동성)

즉, 데이터 변동성 중에서 모델이 설명할 수 있는 비율이에요.

⸻

📌 2. Ridge 회귀 (L2 규제)
	•	문제: 일반 선형회귀는 데이터에 노이즈가 많으면 과적합이 발생할 수 있음.
	•	해결: Ridge는 계수(가중치)의 크기를 제곱해서 패널티를 줌.

\text{손실 함수} = \text{MSE} + \alpha \sum_{j=1}^{p} w_j^2
	•	α (alpha) 값이 크면 규제가 강해짐 → 계수를 더 작게 만듦.
	•	특징: 모든 계수를 0에 가깝게 줄이지만, 완전히 0으로 만들진 않음.
	•	활용: 특성이 많고 다중공선성(multicollinearity)이 있는 경우 성능 안정화에 효과적.

⸻

📌 3. Lasso 회귀 (L1 규제)
	•	Ridge와 달리 계수 절댓값의 합에 패널티를 줌.

\text{손실 함수} = \text{MSE} + \alpha \sum_{j=1}^{p} |w_j|
	•	α가 크면 일부 계수가 완전히 0이 됨 → 자동으로 특성 선택(feature selection) 효과.
	•	특징: 모델이 더 단순해지고, 불필요한 특성을 제거하는 데 유리.
	•	단점: 데이터에 따라 성능이 Ridge보다 불안정할 수 있음.

⸻

📌 Ridge vs Lasso 비교

구분	Ridge (L2)	Lasso (L1)
규제 방식	계수 제곱합 패널티	계수 절댓값 패널티
계수 효과	모두 작아짐 (0에 가깝게)	일부 계수 완전히 0 (특성 제거)
장점	안정적, 과적합 방지	불필요한 특성 제거 (희소성↑)
단점	특성 선택 불가	데이터에 따라 불안정 가능


⸻

✅ 정리하면:
	•	R²: 모델이 데이터를 얼마나 잘 설명하는지 비율
	•	Ridge: 계수 크기 줄이기 (안정적)
	•	Lasso: 불필요한 특성 제거 (해석력↑)

⸻

👉 유진님은 지금 목표가 훈련/테스트 점수를 높이는 것인데,
그럴 경우엔 Ridge → 안정성, Lasso → 불필요한 특성 제거를 병행해서 비교해보면 좋아요.

원하시면 제가 Ridge vs Lasso 성능 비교하는 코드를 바로 짜드릴까요?

## 정답 따라가기

##knn 파트 
- StandardScaler 와 knr함수를 활용함  R2결정계수를 뽑아주는게 knr_test_score
1은 현실적으로는 불가 0.4~0.6이면 잘나온편이다. 훈련점수가 더 높아서 과대적합이 되었다고 한다. 
for 문으로 돌려가면서 하이퍼파라미터 찾기. 
모델의 성능이라고 할 수 있는것은 테스트 점수로 보인다. 
append 로 k 를 함께 넣어 표시해줌 --> 여기서 k max 찾을때 key =lambda x: x[1]: 튜플의 뒤에거 기준으로 찾아라.  1~30 까지 돌렸을 떄 최선의 값을 알려줌 >> 0.340 wjdeh skdha 
<선형회귀>
가장 기초 선형회귀 > 거리기준이 아니면 스케일링 필요없어
하이퍼파라미터 조정도 필요가 없음. 
여러 모델을 비교하려면 테스트 셋을 보는게 맞음> 어느게 실전에서 더 효과가 있었는가! 를 보여주기 때문 
- 예측 실제 값계산 (시각화)>> 테스트 셋의 예측한 결과를 표시 

- 다항 회귀 
여기서는 스케일링이 필요함.. 왜? >> 같은 스케일에 넣어야 공정한 데이터이기 때문(서로 다른 애들이 같이 합쳐질 거라서) 
다항식 특성 생성 fit and transform 으로 바꾸면 -3 으로 고정된 상태이다. 



- 모델 훈련 

